In [1]:
def clean_speech(text):
    stop = set(stopwords.words('english'))
    exclude = set(string.punctuation)
    lemma = WordNetLemmatizer()

    stop_free = ' '.join([word for word in text.lower().split() if word not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = ' '.join([lemma.lemmatize(word) for word in punc_free.split()])

    return normalized.split()

In [2]:
# Use this as a helper file...add all the helper methods or heavy-lifting stuff in this disorganized file.
import requests
import json
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer as detokenizer
from nltk import ngrams
import matplotlib.pyplot as plt 
import pandas as pd
import pickle
#from collections import OrderedDict as OD

import gensim #the library for Topic modelling
from gensim.models.ldamulticore import LdaMulticore
from gensim import corpora, models
import pyLDAvis
import pyLDAvis.gensim_models as viz #LDA visualization library

import string
from nltk.stem.wordnet import WordNetLemmatizer

import warnings
warnings.simplefilter('ignore')
from itertools import chain

# Open JSON file
f = open('speeches.json')

data = list(json.load(f))

# Iterating through the json list
speeches = []
i = 0
for i in range(len(data)):

    speeches.append(data[i])

    i += 1

# Close file
f.close()




num_topics = 4
speeches_df = pd.DataFrame(speeches) 
df = speeches_df.drop(['title', 'date', 'url'], axis=1)
df['clean_transcript'] = df['transcript'].apply(clean_speech)

#create speech words dictionary (contains all unique words from all speeches)
dictionary = corpora.Dictionary(df['clean_transcript'])

#create bag of words or document term matrix
bag_of_words = [dictionary.doc2bow(doc) for doc in df['clean_transcript'] ]

# Instantiate LDA model
lda = gensim.models.ldamodel.LdaModel

# Fit LDA model on the dataset
ldamodel = lda(bag_of_words, num_topics = num_topics, id2word = dictionary, passes=50, minimum_probability = 0)
topics = ldamodel.print_topics(num_topics = num_topics)

# Visualize in jupyter notebook for topic name generation by observing words in clusters
lda_display = viz.prepare(ldamodel, bag_of_words, dictionary, sort_topics = False, mds='mmds')
pyLDAvis.display(lda_display)

# Assign the topics to the speeches
lda_corpus = ldamodel[bag_of_words]

# for doc in lda_corpus:
#     print(doc)

scores = list(chain(*[[score for topic_id, score in topic] \
                  for topic in [doc for doc in lda_corpus]]))

# Calculate threshold to be used to determine which clusters a speech belongs to. Average of probabilities for cluster assignments.
threshold = sum(scores)/len(scores)

# Clusters/Topics/Themes (i.e 0,1,2,3...n) consist of lists of IDs of speeches with similar themes. Ids can be used to identify speech/president in speeches_df. 
clusters = {}
for cluster_id in range(num_topics):
    clusters[cluster_id] = [j for i,j in zip(lda_corpus,df.index) if i[cluster_id][1] > threshold]


#print(clusters)
pyLDAvis.display(lda_display)




/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationW